In [1]:
import numpy as np
from PIL import Image
from skimage.transform import rescale
from skimage import measure as sk_measure
import itertools
import math
import time
from datetime import datetime as dt
import collections

def get_input(day, split=None, f=str.strip):
    fin = f'dat/2021-{day}.txt'
    if split is None:
        input = open(fin, 'r').readlines()
    else:
        input = open(fin, 'r').read().split(split)
    
    if f is not None:
        input = [f(x) for x in input]
    
    return input

def timestamp(ts):
    date = dt.fromtimestamp(ts)
    return f'[{date.hour:02}:{date.minute:02}:{date.second:02}]'

def output(*args, ts=0):
    out = ' '.join([str(x) for x in args])
    if ts:
        ts = timestamp(ts)
        sz = 76 - len(out) - len(ts)
        if sz > 0:
            out += ' ' * sz
        out += ts
    print(out)

def as_ints(input):    
    nums = []
    for x in input:
        x = [int(c) for c in x]
        nums.append(np.asarray(x))
    return np.asarray(nums)

# Day 1

In [2]:
input = get_input(1, f=int)

def depth_inc(lst, win=1):
    cnt = 0
    for i in range(len(lst) - win):
        if lst[i+win] > lst[i]:
            cnt += 1
    return cnt

output(depth_inc(input), ts=1638334897)
output(depth_inc(input, 3), ts=1638335082)

1688                                                              [05:01:37]
1728                                                              [05:04:42]


# Day 2

In [3]:
input = get_input(2)

def find_pos(lst, use_aim=True):
    depth = pos = aim = 0
    for item in lst:
        cmd, X = item.split()
        X = int(X)
        if cmd == 'forward':
            pos += X
            if use_aim:
                depth += X * aim
        elif cmd == 'down':
            if use_aim:
                aim += X
            else:
                depth += X
        elif cmd == 'up':
            if use_aim:
                aim -= X
            else:
                depth -= X
    return depth * pos

output(find_pos(input, False), ts=1638421338)
output(find_pos(input, True), ts=1638421474)

1670340                                                           [05:02:18]
1954293920                                                        [05:04:34]


# Day 3

In [4]:
input = get_input(3)

cnt = [0] * (len(input[0]))
for x in input:
    for i,v in enumerate(x):
        cnt[i] += int(v)

sz = len(input) / 2
gamma = epsilon = ''
for x in cnt:
    if x > sz:
        gamma += '1'
        epsilon += '0'
    else:
        gamma += '0'
        epsilon += '1'
gamma = int(gamma, 2)
epsilon = int(epsilon, 2)

output(gamma*epsilon, ts=1638508079) #1 3969000


def bin_search(lst, pos, most=True):
    cnt = 0
    ones = []
    zeros = []
    for x in lst:
        if int(x[pos]):
            cnt += 1
            ones.append(x)
        else:
            zeros.append(x)
    
    if cnt >= len(lst) / 2:
        if most:
            out = ones
        else:
            out = zeros
    elif most:
        out = zeros
    else:
        out = ones
    
    if len(out) == 1:
        return int(out[0], 2)
    
    return bin_search(out, pos+1, most)


o2  = bin_search(input, 0, True)
co2 = bin_search(input, 0, False)
output(o2 * co2, ts=1638508559) #2 4267809

3969000                                                           [05:07:59]
4267809                                                           [05:15:59]


# Day 4

In [5]:
input = get_input(4)

calls = [int(x) for x in input[0].split(',')]

boards = []
for i in range(2, len(input)-4, 6):
    b = []
    for j in range(5):
        b.append([int(x) for x in input[i+j].split()])
    boards.append(np.asarray(b))

def bingo(board):
    for row in board:
        if np.sum(row) == -5:
            return True
    for col in board.T:
        if np.sum(col) == -5:
            return True
    return False

def score(board, call):
    board[np.where(board==-1)] = 0
    return np.sum(board) * call

cnt = 0
last = len(boards)
for x in calls:
    hit = False
    for b in boards:
        if x in b:
            b[np.where(b==x)] = -1
            if bingo(b):
                cnt += 1
                if cnt == 1:
                    output(score(b,x), ts=1638594977) #1
                elif cnt == last:
                    output(score(b,x), ts=1638595294) #2
                b[:][:] = -99

8580                                                              [05:16:17]
9576                                                              [05:21:34]


# Day 5

In [6]:
def f(s):
    a, _, b = s.split()
    x1, y1 = [int(i) for i in a.split(',')]
    x2, y2 = [int(i) for i in b.split(',')]  
    return x1, y1, x2, y2

input = get_input(5, f=f)

vents = np.zeros((1000,1000), np.int32)
cnt = np.ones((1000,1000), np.int32)

for x1, y1, x2, y2 in input:
    if x1 == x2:
        if y1 > y2:
            r = range(y2, y1+1)
        else:
            r = range(y1, y2+1)
        for y in r:
            vents[x1][y] += 1
    elif y1 == y2:
        if x1 > x2:
            r = range(x2, x1+1)
        else:
            r = range(x1, x2+1)
        for x in r:
            vents[x][y1] += 1

output(np.sum(cnt[np.where(vents > 1)]), ts=1638681085) #1

for x1, y1, x2, y2 in input:
    if x1 != x2 and y1 != y2:
        if x1 < x2:
            r = range(x1, x2+1)
        else:
            r = range(x1, x2-1, -1)
        
        if y1 < y2:
            f = lambda a,b: a+b
        else:
            f = lambda a,b: a-b
        
        for y,x in enumerate(r):
            vents[x][f(y1,y)] += 1

output(np.sum(cnt[np.where(vents > 1)]), ts=1638682511) #2

7436                                                              [05:11:25]
21104                                                             [05:35:11]


# Day 6

In [7]:
input = get_input(6, split=',', f=int)

fish = collections.Counter(input) # count fish at each time

def epoch(a):
    b = {}              # next epoch
    a[7] = a[7] + a[0]  # reset 0 -> 6 (subtract below)
    b[8] = a[0]         # add new fish (not subtracted)
    for i in range(8):  # subtract 1 from each
        b[i] = a[i+1]
    return b

for day in range(80):
    fish = epoch(fish)

assert sum(fish.values()) == 350149
output(sum(fish.values()), ts=1638767460) #1

for day in range(256-80):
    fish = epoch(fish)
    
assert sum(fish.values()) == 1590327954513
output(sum(fish.values()), ts=1638768282) #2

350149                                                            [05:11:00]
1590327954513                                                     [05:24:42]


# Day 7

In [8]:
input = get_input(7, split=',', f=int)

low_lin = low_geom = 10**20
for i in range(max(input)):
    cost_lin = cost_geom = 0
    for x in input:
        cost_lin += abs(x-i)
        d = abs(x-i)
        cost_geom += (d*d+d)//2
    if cost_lin < low_lin:
        low_lin = cost_lin
    if cost_geom < low_geom:
        low_geom = cost_geom
        
output(low_lin, ts=1638853498)
output(low_geom, ts=1638853947)

356992                                                            [05:04:58]
101268110                                                         [05:12:27]


# Day 8

In [9]:
input = get_input(8)

outs = [x.split('|')[1] for x in input]

cnt = 0
for x in outs:
    for y in x.split():
        if len(y) in [2,3,4,7]:
            cnt += 1

assert cnt == 365
output(cnt, ts=1638940069) #1

cnt = 0
for x in input:
    ins, outs = x.split('|')
    ins = ins.split()
    a = b = c = d = e = f = g = ''
    
    # find easy nums by length
    for y in ins:
        if len(y) == 2:
            one = y
        elif len(y) == 3:
            seven = y
        elif len(y) == 4:
            four = y
        elif len(y) == 7:
            eight = y
    # find three and six using one
    for y in ins:
        if len(y) == 5 and one[0] in y and one[1] in y:
            three = y
        elif len(y) == 6 and not (one[0] in y and one[1] in y):
            six = y
    # a - in seven but not one
    for i in seven:
        if i not in one:
            a = i
    # b - in four but not three
    for i in four:
        if i not in three:
            b = i
    # c - in eight but not six
    for i in eight:
        if i not in six:
            c = i
    # d - in three and four but not one
    # g - in three but not four (and not a)
    for i in three:
        if i in four and i not in one:
            d = i
        elif i not in four and i != a:
            g = i
    # e - in six but not three (and not b)
    # f - in six and one
    for i in six:
        if i not in three and i != b:
            e = i
        elif i in one:
            f = i
    
    if '' in [a,b,c,d,e,f,g]:
        print('shiza', ins)
        continue
    
    out = ''
    for y in outs.split():
        if len(y) == 6 and d not in y:
            out += '0'
        elif len(y) == 2:
            out += '1'
        elif len(y) == 5 and e in y:
            out += '2'
        elif len(y) == 5 and c in y and f in y:
            out += '3'
        elif len(y) == 4:
            out += '4'
        elif len(y) == 5 and b in y:
            out += '5'
        elif len(y) == 6 and c not in y:
            out += '6'
        elif len(y) == 3:
            out += '7'
        elif len(y) == 7:
            out += '8'
        elif len(y) == 6 and e not in y:
            out += '9'
        else:
            print('shiza', y)
    #print(out)
    
    cnt += int(out)

assert cnt == 975706
output(cnt, ts=1638942090) #2

365                                                               [05:07:49]
975706                                                            [05:41:30]


# Day 9

In [10]:
input = get_input(9)

nums = []
for x in input:
    x = [int(c) for c in x]
    nums.append(x)
nums = np.asarray(nums)

w = len(nums[0])
h = len(nums)
cnt = 0
for y,row in enumerate(nums):
    for x,val in enumerate(row):
        if x < w-1 and val >= nums[y][x+1]: continue
        if x > 0 and val >= nums[y][x-1]: continue
        if y < h-1 and val >= nums[y+1][x]: continue
        if y > 0 and val >= nums[y-1][x]: continue
        cnt += val + 1

output(cnt, ts=1639026784) #1

# or...
#from skimage import morphology
#minima = nums[np.where(morphology.local_minima(nums))]
#cnt = sum(minima) + len(minima)

# basins are bordered by 9s or edges, everything else is equiv
nums[np.where(nums < 9)] = 1
# label each basin as a unique group
img = sk_measure.label(nums, connectivity=1)
# mask out the 9s borders to leave just basins
img[np.where(nums == 9)] = 0
# let skimage do the heavy lifting
props = sk_measure.regionprops(img)
# get the largest basins
basins = sorted([p.area for p in props], reverse=True)
out = 1
for b in sorted(basins, reverse=True)[:3]:
    out *= b

assert out == 1317792
output(out, ts=1639028100)

436                                                               [05:13:04]
1317792                                                           [05:35:00]


# Day 10

In [11]:
input = get_input(10)

def points(x):
    if x == ')':return 3
    if x == ']':return 57
    if x == '}':return 1197
    if x == '>':return 25137

def points2(x):
    if x == ')':return 1
    if x == ']':return 2
    if x == '}':return 3
    if x == '>':return 4

closure = {
    '<': '>',
    '[': ']',
    '{': '}',
    '(': ')'
}

errors = 0
total = []
for line in input:
    expected = ''
    score = 0
    for x in line:
        if x in closure:
            expected += closure[x]
        elif x != expected[-1]:
            score += points(x)
            break
        else:
            expected = expected[:-1]
    
    if score > 0:
        errors += score
    else:
        for x in reversed(expected):
            score *= 5
            score += points2(x)
        total.append(score)    

assert errors == 216297
output(errors, ts=1639113303)

assert np.median(total) == 2165057169
output(np.median(total), ts=1639113738)

216297                                                            [05:15:03]
2165057169.0                                                      [05:22:18]


# Day 11

In [12]:
input = as_ints(get_input(11))

def flash(x, y, arr):
    w, h = arr.shape
    if x > 0: arr[y][x-1] += 1
    if x > 0 and y > 0: arr[y-1][x-1] += 1
    if y > 0: arr[y-1][x] += 1
    if y > 0 and x < w-1: arr[y-1][x+1] += 1
    if x < w-1: arr[y][x+1] += 1
    if x < w-1 and y < h-1: arr[y+1][x+1] += 1
    if y < h-1: arr[y+1][x] += 1
    if y < h-1 and x > 0: arr[y+1][x-1] += 1

def octo_step(arr):
    mask = np.zeros(arr.shape, np.int32)
    arr += 1
    hit = True
    while hit:
        hit = False
        flashes = np.where(arr > 9)
        for y,x in zip(flashes[0], flashes[1]):
            if mask[y][x] == 0:
                hit = True
                flash(x, y, arr)
        mask[flashes] = 1
    arr[np.where(mask)] = 0
    #print(arr)
    return np.sum(mask)

cnt = 0
for i in range(100):
    cnt += octo_step(input)
output(cnt) #1

cnt = 100
for i in range(10000):
    cnt += 1
    if octo_step(input) == input.size:
        break
output(cnt) #2

1691
216


# Day 12

In [13]:
input = get_input(12)

input = [x.split('-') for x in input]

def next_cave(cave, path, allow_two):
    p = None
    can_go = False
    can_go = cave.isupper() or cave not in path
    if cave.isupper():
        can_go = True
    elif cave == 'start':
        can_go = False
    elif cave not in path:
        can_go == True
    elif allow_two:
        caves = [x for x in path if x.islower()]
        if max(collections.Counter(caves).values()) == 1:
            can_go = True
    
    if can_go:
        p = path.copy()
        p.append(cave)
    return p

def extend_path(cave, path, allow_two):
    paths = []
    for a,b in input:
        if a == cave:
            p = next_cave(b, path, allow_two)
            if p:
                paths.append(p)
        elif b == cave:
            p = next_cave(a, path, allow_two)
            if p:
                paths.append(p)
    return paths
    
def search(paths, allow_two=False):
    if len(paths) == 0:
        cave = 'start'
        path = [cave]
        return extend_path(cave, path, allow_two)
    
    new_paths = []
    for p in paths:
        cave = p[-1]
        if cave == 'end':
            new_paths.append(p)
        else:
            new_paths.extend(extend_path(cave, p, allow_two))
    return new_paths
    
paths = search([])
while True:
    paths = search(paths)
    ends = set(x[-1] for x in paths)
    if len(ends) == 1 and 'end' in ends: break

#for p in paths:print(p)
    
output(len(paths)) #1

paths = search([])
while True:
    paths = search(paths, True)
    ends = set(x[-1] for x in paths)
    if len(ends) == 1 and 'end' in ends: break

#for p in paths:print(p)
    
output(len(paths)) #2

4775
152480


# Day 13

In [14]:
input = get_input(13)

dots = []
folds = []
for x in input:
    if not x:
        continue
    elif 'fold' in x:
        folds.append(x.split()[2].split('='))
    else:
        dots.append([int(c) for c in x.split(',')])
        
w = max([x[0] for x in dots]) + 1
h = max([x[1] for x in dots]) + 1
if w % 2 == 0: w += 1
if h % 2 == 0: h += 1

arr = np.zeros((h,w), np.int32)
for x,y in dots:
    arr[y][x] = 1

dots = arr

#print(dots)
for d, loc in folds:
    loc = int(loc)
    #print(d,loc,dots.shape,loc*2+1)
    if d == 'y':
        for y in range(loc):
            for x in range(w):
                dots[y][x] = dots[y][x] or dots[h-y-1][x]
        h = loc
        dots = dots[:h,:]
    else:
        for y in range(h):
            for x in range(loc):
                dots[y][x] = dots[y][x] or dots[y][w-x-1]
        w = loc
        dots = dots[:,:w]

    #print(dots)
    #break

output(np.sum(dots))
for row in dots:
    line = [str(x) for x in row]
    print(''.join(line).replace('1','#').replace('0',' '))

103
###  #  #  ##  #    ###   ##  ###   ##  
#  # #  # #  # #    #  # #  # #  # #  # 
#  # #### #  # #    #  # #    #  # #  # 
###  #  # #### #    ###  #    ###  #### 
# #  #  # #  # #    # #  #  # # #  #  # 
#  # #  # #  # #### #  #  ##  #  # #  # 


# Day 14

In [89]:
input = get_input(14)

chain = input[0]
rules = {x.split(' -> ')[0]: x.split(' -> ')[1] for x in input[2:]}

#print('start',dt.utcnow())

def react(chain):
    pairs = [f'{chain[i]}{chain[i+1]}' for i in range(len(chain)-1)]
    new_chain = ''
    for p in pairs:
        if p in rules:
            new_chain += p[0] + rules[p]
        else:
            new_chain += p[0]
    new_chain += chain[-1]
    return new_chain

chains = [chain]
for i in range(10):
    #print(i,chain,len(chain))
    chain = react(chain)
    for c in chains:
        if c in chain:print(i)
    chains.append(chain)
#printchain,len(chain))

count = collections.Counter(chain)
output(max(count.values()) - min(count.values())) #1

'''
react_20 = {}
for pair in rules:
    chain = pair
    for i in range(20):
        chain = react(chain)
    react_20[pair] = chain

cnt = {}
for key,val in react_20.items():
    cnt[key] = collections.Counter(val)
    cnt[key][val[-1]] -= 1
print('react',dt.utcnow())

out = {key:0 for key in set(react(input[0]))}

def do_20(chain):
    out = ''
    for i in range(len(chain)-1):
        pair = chain[i] + chain[i+1]
        out += react_20[pair][:-1]
    return out + chain[-1]

chain = do_20(input[0])
print('do_20',dt.utcnow())

for i in range(len(chain)-1):
    pair = chain[i] + chain[i+1]
    for k in out.keys():
        out[k] += cnt[pair][k]
out[chain[-1]] += 1

print('count',dt.utcnow())
out = {k:v for k,v in out.items() if v > 0}
print(out)
output(max(out.values()) - min(out.values())) #2
'''
output(3816397135460)

2584
3816397135460


In [91]:
input = get_input(14)

chain = input[0]
rules = {x.split(' -> ')[0]: x.split(' -> ')[1] for x in input[2:]}
for a,b in rules.items():
    rules[a] = (a[0]+b,b+a[1])

pairs = {k:0 for k in rules.keys()}
for i in range(len(chain)-1):
    pairs[chain[i:i+2]] += 1

def react(pairs):
    new_pairs = {k:0 for k in pairs.keys()}
    for pair,cnt in pairs.items():
        for p in rules[pair]:
            new_pairs[p] += cnt
    return new_pairs
for i in range(10):
    pairs = react(pairs)

counts = {k:0 for k in set([x[0] for x in rules.keys()])}
for pair, cnt in pairs.items():
    counts[pair[0]] += cnt
counts[chain[-1]] += 1
output(max(counts.values()) - min(counts.values()))

for i in range(30):
    pairs = react(pairs)

counts = {k:0 for k in set([x[0] for x in rules.keys()])}
for pair, cnt in pairs.items():
    counts[pair[0]] += cnt
counts[chain[-1]] += 1
output(max(counts.values()) - min(counts.values()))

2584
3816397135460


# Day 15

In [95]:
input = get_input(15)

h = len(input)
w = len(input[0])

arr = np.zeros((h,w),np.int32)
for y,row in enumerate(input):
    for x,val in enumerate(row):
        arr[y][x] = int(val)

class Path(object):
    def __init__(self, cost=0, node=None):
        self.cost = cost
        self.node = node

def to_key(x,y):
    return f'{x},{y}'

costs = {}
def find_path2(x, y, cost, arr, w, h):
    cost += arr[y][x]
    key = to_key(x,y)
    if key not in costs or (cost < costs[key]):
        costs[key] = cost
        p = Path(cost, (x,y))
        find_path(p, arr, w, h)

def find_path(path, arr, w, h):
    x,y = path.node
    key = to_key(x,y)
    if key in costs and costs[to_key(x,y)] != path.cost:
        # path was pruned
        return
    
    if x < w-1:
        find_path2(x+1, y, path.cost, arr, w, h)
    if y < h-1:
        find_path2(x, y+1, path.cost, arr, w, h)
                
        
p = Path(0, (0,0))
find_path(p, arr, w, h)
output(costs[to_key(w-1,h-1)]) #1 707

arr_large = np.zeros((h*5,w*5), np.int32)

for y in range(5):
    for x in range(5):
        block = arr.copy()
        block += y + x
        block[np.where(block > 9)] -= 9
        arr_large[h*y:h*(y+1),w*x:w*(x+1)] = block

costs = {}
p = Path(0, (0,0))
find_path(p, arr_large, w*5, h*5)
output(costs[to_key(w*5-1,h*5-1)]) #2
#NOT 2948

40
315


In [ ]:
def find_route(start, target, steps, best):
    q = [(steps, start[0], start[1])]
    while q:
        q = sorted(q, key = lambda x: x[0])
        steps, x, y = q.pop(0)
        if (x,y) in best and best[(x,y)] <= steps:
            continue
        best[(x,y)] = min(best[(x,y)] if (x,y) in best else steps, steps)
        if (x,y) == target:
            return best[(x,y)]
        for a, b in [coord for coord in ((x,y+1), (x-1,y), (x+1,y), (x,y-1)) if coord in grid]:
            q.append((steps + grid[(a,b)], a, b))

input = get_input(15)

data = [[int(x) for x in y] for y in input]
grid = {(x,y) : data[y][x] for x in range(len(data[0])) for y in range(len(data))}
print(find_route((0,0), (len(data[0]) - 1, len(data) - 1), 0, {}))
for z in [lambda: (x + i * len(data[0]), y), lambda: (x, y + i * len(data))]:
    new_coords = {}
    for x,y in grid:
        for i in range(1,5):
            new_coords[z()] = (grid[(x,y)] + i) % 10 + 1 if (grid[(x,y)] + i) > 9 else (grid[(x,y)] + i)
    grid.update(new_coords)
print(find_route((0,0), (len(data[0])*5 - 1, len(data)*5 - 1), 0, {}))

707


# Day 16

In [25]:
input = get_input(16)

msg = bin(int(input[0], 16))[2:]
exp_len = len(input[0]) * 4
msg = '0' * (exp_len - len(msg)) + msg

version_sum = 0

def product(nums):
    prod = 1
    for x in nums:
        prod *= x
    return prod

def greater_than(nums):
    a, b = nums
    if a > b:
        return 1
    return 0

def less_than(nums):
    a, b = nums
    if a < b:
        return 1
    return 0

def equal_to(nums):
    a, b = nums
    if a == b:
        return 1
    return 0

def parse_hex(msg):
    global version_sum
    if not msg:
        return
    
    ver = int(msg[:3], 2)
    typ = int(msg[3:6], 2)
    #print(f'Version: {ver}, Type ID: {typ}')
    version_sum += ver
    
    if typ == 4:
        i = 6
        num = ''
        while True:
            block = msg[i:i+5]
            num += block[1:]
            i += 5
            if block[0] == '0':
                break
        #print('Literal', int(num, 2))
        return int(num, 2), i
    else:
        if typ == 0:
            func = sum
        elif typ == 1:
            func = product
        elif typ == 2:
            func = min
        elif typ == 3:
            func = max
        elif typ == 5:
            func = greater_than
        elif typ == 6:
            func = less_than
        elif typ == 7:
            func = equal_to
            
        len_id = msg[6]
        nums = []
        #print('Control',len_id)
        if len_id == '0':
            num_bits = int(msg[7:22], 2)
            bits_read = 0
            i = 22
            while bits_read < num_bits:
                num, j = parse_hex(msg[i:])
                #print(num, j)
                nums.append(num)
                bits_read += j
                i += j
            if bits_read != num_bits:
                print(f'WARN: Expected {num_bits} but read {bits_read}')
            return func(nums), i
        else:
            num_pkts = int(msg[7:18], 2)
            #print('Packets:',num_pkts)
            i = 18
            for p in range(num_pkts):
                num, j = parse_hex(msg[i:])
                #print(num, j)
                nums.append(num)
                i += j
            return func(nums), i
                
#print(msg)
res = parse_hex(msg)
output(version_sum) #1
output(res[0]) #2

969
124921618408


# Day 17

In [74]:
# target area: x=88..125, y=-157..-103
target_x = (88,125)
target_y = (-157, -103)

# Test
#target_x = (20,30)
#target_y = (-10,-5)

def in_target(pos):
    x, y = pos
    min_x, max_x = target_x
    min_y, max_y = target_y
    return x >= min_x and x <= max_x and y >= min_y and y <= max_y

def past_target(pos):
    x, y = pos
    min_x, max_x = target_x
    min_y, max_y = target_y
    return x > max_x or y < min_y

def step(pos, vx, vy):
    pos[0] += vx
    pos[1] += vy
    if vx > 0:
        vx -= 1
    elif vx < 0:
        vx += 1
    vy -= 1
    return pos, vx, vy

high_y = high_vx = high_vy = 0
cnt = 0
for vx in range(target_x[1]*2):
    vx0 = vx
    for vy in range(target_y[0],target_y[0]*-2):
        vx = vx0
        pos = [0,0]
        max_y = 0
        vy0 = vy
        for i in range(1000):
            pos, vx, vy = step(pos, vx, vy)
            max_y = max(max_y, pos[1])
            if in_target(pos):
                cnt += 1
                #print(vx0,vy0,max_y)
                if max_y > high_y:
                    high_y = max_y
                    high_vx = vx0
                    high_vy = vy0
                break
            if past_target(pos):
                break
                
print(high_vx, high_vy)
output(high_y) #1
output(cnt) #2

13 156
12246
3528


# Day 18

In [151]:
def find_left(lst, idx=[]):
    if type(lst) != list:
        return idx
    idx.append(1)
    return find_left(lst[1], idx)

def find_right(lst, idx=[]):
    if type(lst) != list:
        return idx
    idx.append(0)
    return find_right(lst[0], idx)
    
def explode(lst, a, b, c, d):
    pair = lst[a][b][c][d]
    
    if d == 1:
        lst[a][b][c][0] += pair[0]
    elif c == 1:
        left = find_left(lst[a][b][0], [])
        if len(left) == 1:
            lst[a][b][0][left[0]] += pair[0]
        elif len(left) == 0:
            lst[a][b][0] += pair[0]
        else:
            print("SHIZA")
    elif b == 1:
        left = find_left(lst[a][0], [])
        if len(left) == 2:
            lst[a][0][left[0]][left[1]] += pair[0]
        elif len(left) == 1:
            lst[a][0][left[0]] += pair[0]
        elif len(left) == 0:
            lst[a][0] += pair[0]
        else:
            print("SHIZA")
    elif a == 1:
        left = find_left(lst[0], [])
        if len(left) == 3:
            lst[0][left[0]][left[1]][left[2]] += pair[0]
        elif len(left) == 2:
            lst[0][left[0]][left[1]] += pair[0]
        elif len(left) == 1:
            lst[0][left[0]] += pair[0]
        elif len(left) == 0:
            lst[0] += pair[0]
        else:
            print("SHIZA")
    
    if d == 0:
        right = find_right(lst[a][b][c][1], [])
        if len(right) == 1:
            lst[a][b][c][1][right[0]] += pair[1]
        elif len(right) == 0:
            lst[a][b][c][1] += pair[1]
        else:
            print("SHIZA")
    elif c == 0:
        right = find_right(lst[a][b][1], [])
        if len(right) == 2:
            lst[a][b][1][right[0]][right[1]] += pair[1]
        elif len(right) == 1:
            lst[a][b][1][right[0]] += pair[1]
        elif len(right) == 0:
            lst[a][b][1] += pair[1]
        else:
            print("SHIZA")
    elif b == 0:
        right = find_right(lst[a][1], [])
        if len(right) == 3:
            lst[a][1][right[0]][right[1]][right[2]] += pair[1]
        elif len(right) == 2:
            lst[a][1][right[0]][right[1]] += pair[1]
        elif len(right) == 1:
            lst[a][1][right[0]] += pair[1]
        elif len(right) == 0:
            lst[a][1] += pair[1]
        else:
            print("SHIZA")
    elif a == 0:
        right = find_right(lst[1], [])
        if len(right) == 4:
            lst[1][right[0]][right[1]][right[2]][right[3]] += pair[1]
        elif len(right) == 3:
            lst[1][right[0]][right[1]][right[2]] += pair[1]
        elif len(right) == 2:
            lst[1][right[0]][right[1]] += pair[1]
        elif len(right) == 1:
            lst[1][right[0]] += pair[1]
        elif len(right) == 0:
            lst[1] += pair[1]
        else:
            print("SHIZA")
    
    lst[a][b][c][d] = 0
    return lst
  
def find_explode(lst):
    a0, b0 = lst
    if type(a0) == list:
        a1, b1 = a0
        if type(a1) == list:
            a2, b2 = a1
            if type(a2) == list:
                a3, b3 = a2
                if type(a3) == list:
                    return explode(lst, 0, 0, 0, 0)
                if type(b3) == list:
                    return explode(lst, 0, 0, 0, 1)
            if type(b2) == list:
                a3, b3 = b2
                if type(a3) == list:
                    return explode(lst, 0, 0, 1, 0)
                if type(b3) == list:
                    return explode(lst, 0, 0, 1, 1)
        if type(b1) == list:
            a2, b2 = b1
            if type(a2) == list:
                a3, b3 = a2
                if type(a3) == list:
                    return explode(lst, 0, 1, 0, 0)
                if type(b3) == list:
                    return explode(lst, 0, 1, 0, 1)
            if type(b2) == list:
                a3, b3 = b2
                if type(a3) == list:
                    return explode(lst, 0, 1, 1, 0)
                if type(b3) == list:
                    return explode(lst, 0, 1, 1, 1)
    if type(b0) == list:
        a1, b1 = b0
        if type(a1) == list:
            a2, b2 = a1
            if type(a2) == list:
                a3, b3 = a2
                if type(a3) == list:
                    return explode(lst, 1, 0, 0, 0)
                if type(b3) == list:
                    return explode(lst, 1, 0, 0, 1)
            if type(b2) == list:
                a3, b3 = b2
                if type(a3) == list:
                    return explode(lst, 1, 0, 1, 0)
                if type(b3) == list:
                    return explode(lst, 1, 0, 1, 1)
        if type(b1) == list:
            a2, b2 = b1
            if type(a2) == list:
                a3, b3 = a2
                if type(a3) == list:
                    return explode(lst, 1, 1, 0, 0)
                if type(b3) == list:
                    return explode(lst, 1, 1, 0, 1)
            if type(b2) == list:
                a3, b3 = b2
                if type(a3) == list:
                    return explode(lst, 1, 1, 1, 0)
                if type(b3) == list:
                    return explode(lst, 1, 1, 1, 1)
    return None    

def split(lst, a, b, c, d):
    if d is not None:
        x = lst[a][b][c][d] / 2
        lst[a][b][c][d] = [math.floor(x), math.ceil(x)]
    elif c is not None:
        x = lst[a][b][c] / 2
        lst[a][b][c] = [math.floor(x), math.ceil(x)]
    elif b is not None:
        x = lst[a][b] / 2
        lst[a][b] = [math.floor(x), math.ceil(x)]
    else:
        x = lst[a] / 2
        lst[a] = [math.floor(x), math.ceil(x)]
        
    return lst

def find_split(lst):
    a0, b0 = lst
    if type(a0) == list:
        a1, b1 = a0
        if type(a1) == list:
            a2, b2 = a1
            if type(a2) == list:
                a3, b3 = a2
                if a3 > 9:
                    return split(lst, 0, 0, 0, 0)
                if b3 > 9:
                    return split(lst, 0, 0, 0, 1)
            elif a2 > 9:
                return split(lst, 0, 0, 0, None)
            if type(b2) == list:
                a3, b3 = b2
                if a3 > 9:
                    return split(lst, 0, 0, 1, 0)
                if b3 > 9:
                    return split(lst, 0, 0, 1, 1)
            elif b2 > 9:
                return split(lst, 0, 0, 1, None)
        elif a1 > 9:
            return split(lst, 0, 0, None, None)
        if type(b1) == list:
            a2, b2 = b1
            if type(a2) == list:
                a3, b3 = a2
                if a3 > 9:
                    return split(lst, 0, 1, 0, 0)
                if b3 > 9:
                    return split(lst, 0, 1, 0, 1)
            elif a2 > 9:
                return split(lst, 0, 1, 0, None)
            if type(b2) == list:
                a3, b3 = b2
                if a3 > 9:
                    return split(lst, 0, 1, 1, 0)
                if b3 > 9:
                    return split(lst, 0, 1, 1, 1)
            elif b2 > 9:
                return split(lst, 0, 1, 1, None)
        elif b1 > 9:
            return split(lst, 0, 1, None, None)
    elif a0 > 9:
        return split(lst, 0, None, None, None)
    if type(b0) == list:
        a1, b1 = b0
        if type(a1) == list:
            a2, b2 = a1
            if type(a2) == list:
                a3, b3 = a2
                if a3 > 9:
                    return split(lst, 1, 0, 0, 0)
                if b3 > 9:
                    return split(lst, 1, 0, 0, 1)
            elif a2 > 9:
                return split(lst, 1, 0, 0, None)
            if type(b2) == list:
                a3, b3 = b2
                if a3 > 9:
                    return split(lst, 1, 0, 1, 0)
                if b3 > 9:
                    return split(lst, 1, 0, 1, 1)
            elif b2 > 9:
                return split(lst, 1, 0, 1, None)
        elif a1 > 9:
            return split(lst, 1, 0, None, None)
        if type(b1) == list:
            a2, b2 = b1
            if type(a2) == list:
                a3, b3 = a2
                if a3 > 9:
                    return split(lst, 1, 1, 0, 0)
                if b3 > 9:
                    return split(lst, 1, 1, 0, 1)
            elif a2 > 9:
                return split(lst, 1, 1, 0, None)
            if type(b2) == list:
                a3, b3 = b2
                if a3 > 9:
                    return split(lst, 1, 1, 1, 0)
                if b3 > 9:
                    return split(lst, 1, 1, 1, 1)
            elif b2 > 9:
                return split(lst, 1, 1, 1, None)
        elif b1 > 9:
            return split(lst, 1, 1, None, None)
    elif b0 > 9:
        return split(lst, 1, None, None, None)
        
    return None
                    
def add(a, b):
    return [a, b]

def reduce(lst):
    #print(lst)
    out = find_explode(lst)
    if out is not None:
        #print('explode')
        return reduce(out)
    out = find_split(lst)
    if out is not None:
        #print('split')
        return reduce(out)
    return lst

def magnitude(lst):
    a, b = lst
    if type(a) == list:
        a = magnitude(a)
    if type(b) == list:
        b = magnitude(b)
    return 3*a + 2*b

import ast
input = get_input(18, f=ast.literal_eval)
x = input[0]
for y in input[1:]:
    #print(x,y)
    x = add(x, y)
    x = reduce(x)
    #print(x)
    
output(magnitude(x)) #1

import copy

max_mag = 0
input = get_input(18, f=ast.literal_eval)
for x in input:
    for y in input:
        if x != y:
            a = add(copy.deepcopy(x), copy.deepcopy(y))
            b = reduce(a)
            c = magnitude(b)
            max_mag = max(max_mag, c)
output(max_mag) #2

3051
4812


# Day 19

In [101]:
input = get_input(19)

class Scanner(object):
    def __init__(self):
        self.x = 0
        self.y = 1
        self.z = 2
        self.beacons = []
    
    def add_beacon(self, b):
        self.beacons.append(b)
        
    def finalize(self):
        self.beacons = np.sort(np.asarray(self.beacons, np.int8))
        #print(self.beacons)
        self.transform()
    
    def transform(self):
        self.xforms = []
        negs = self.negate(self.beacons)
        for n in negs:
            self.xforms.extend(self.rotate(n))
        self.xforms = negs
    
    def rotate(self, beacons):
        lst = []
        for b in beacons:
            xform = []
            rot = ((2,1,0), (0,2,1), (1,0,2))
            for x,y,z in rot:
                a = np.zeros(b.shape, np.int8)
                a[0] = b[x].copy()
                a[1] = b[y].copy()
                a[2] = b[z].copy()
                xform.append(a)
            lst.append(xform)
        return np.sort(lst)
            
    
    def negate(self, beacons):
        lst = []
        for b in beacons:
            xform = []
            neg = ((0,), (1,), (2,), (0,1), (0,2), (1,2), (0,1,2))
            for ords in neg:
                a = b.copy()
                for o in ords:
                    a[o] *= -1
                xform.append(a)
            xform.append(b)
            lst.append(xform)
        return np.sort(lst)
    
scanners = []
s = None
for line in input:
    if not line:
        continue
    elif line.startswith('---'):
        if s is not None:
            scanners.append(s)
        s = Scanner()
    else:
        s.add_beacon([int(x) for x in line.split(',')])
scanners.append(s)

for s in scanners:
    s.finalize()
    #print(s.beacons)
    
a = np.asarray([[-618,-824,-621],
[-537,-823,-458],
[-447,-329,318],
[404,-588,-901],
[544,-627,-890],
[528,-643,409],
[-661,-816,-575],
[390,-675,-793],
[423,-701,434],
[-345,-311,381],
[459,-707,401],
[-485,-357,347]])

b = np.asarray([[686,422,578],
[605,423,415],
[515,917,-361],
[-336,658,858],
[-476,619,847],
[-460,603,-452],
[729,430,532],
[-322,571,750],
[-355,545,-477],
[413,935,-424],
[-391,539,-444],
[553,889,-390]])

s0 = scanners[0].beacons
s1 = scanners[1].beacons

s=scanners[0]
print(len(s.beacons), len(s.xforms), len(s.xforms[0]))
print(s.xforms)

for b1 in s1:
    for b0 in s0:
        offset = b0 - b1
        tmp = s0 - s1 + offset
        if offset[1] == -1246:
            print(offset)
            for i in range(3):
                c = collections.Counter(tmp[:,i])
                print(c)

#a = np.sort(a)
b[:,0] *= -1
b[:,2] *= -1
off = a[0] - b[0]
b += (a[0]-b[0])
#b = np.sort(b)
print(a-b)

#...?

25 25 8
[[[ -76  108  123]
  [-108   76  123]
  [-123 -108  -76]
  [  76  108  123]
  [-123  -76  108]
  [-123 -108   76]
  [-123   76  108]
  [-108  -76  123]]

 [[  16  103  125]
  [-103  -16  125]
  [-125 -103   16]
  [ -16  103  125]
  [-125   16  103]
  [-125 -103  -16]
  [-125  -16  103]
  [-103   16  125]]

 [[ -34   70   79]
  [ -70   34   79]
  [ -79  -70  -34]
  [  34   70   79]
  [ -79  -34   70]
  [ -79  -70   34]
  [ -79   34   70]
  [ -70  -34   79]]

 [[ -25   93  122]
  [-122   25   93]
  [-122  -93  -25]
  [  25   93  122]
  [ -93  -25  122]
  [-122  -93   25]
  [ -93   25  122]
  [-122  -25   93]]

 [[ -25   54   55]
  [ -55   25   54]
  [ -55  -54  -25]
  [  25   54   55]
  [ -54  -25   55]
  [ -55  -54   25]
  [ -54   25   55]
  [ -55  -25   54]]

 [[  27   91  101]
  [-101  -27   91]
  [-101  -91   27]
  [ -27   91  101]
  [ -91   27  101]
  [-101  -91  -27]
  [ -91  -27  101]
  [-101   27   91]]

 [[ -55   89  125]
  [ -89   55  125]
  [-125  -89  -55]
  [  55   8

# Day 20

In [80]:
input = get_input(20)

alg = input[0].replace('.', '0').replace('#', '1')
alg = [int(x) for x in alg]

h = len(input) - 2
w = len(input[2])

img = np.zeros((h,w), np.int8)
for y,row in enumerate(input[2:]):
    for x,c in enumerate(row):
        if c == '#':
            img[y][x] = 1
    
def enhance(img, ones):
    #ones = False #test input
    if ones:
        arr = np.ones
    else:
        arr = np.zeros
    h, w, = img.shape
    h += 4
    w += 4
    img_out = arr((h,w), np.int8)
    img_in = arr((h,w), np.int8)
    img_in[2:-2,2:-2] = img
    for y in range(1,h-1):
        for x in range(1,w-1):
            kernel = img_in[y-1:y+2,x-1:x+2]
            idx = ''
            for k in kernel.flat:
                idx += str(k)
            idx = int(idx, 2)
            img_out[y][x] = alg[idx]
    return img_out[1:-1,1:-1], not ones


ones = False
for i in range(2):
    img, ones = enhance(img, ones)
output(np.sum(img)) #1

for i in range(48):
    img, ones = enhance(img, ones)
output(np.sum(img)) #2

5884
19043


In [47]:
input = get_input(25)

w = len(input[0])
h = len(input)

arr = np.zeros((h,w), np.int8)

for y in range(h):
    for x in range(w):
        if input[y][x] == '>':
            arr[y][x] = 1
        elif input[y][x] == 'v':
            arr[y][x] = 2

def print_cuc(arr):
    print(' ' + str(arr).replace('0','.').replace('1','>').replace('2','v').replace('[','').replace(']','') + '\n')
    
def one_move(arr, m, n, val):
    arr[m][n] = val
    if val == 1:
        n -= 1
        if n < 0:
            n = w-1
    else:
        m -= 1
        if m < 0:
            m = h-1
    arr[m][n] = 0
            
def move(arr, n):
    out = arr.copy()
    y, x = np.where(arr==n)
    if n == 1:
        dim = x
        lim = w
    else:
        dim = y
        lim = h
    
    dim += 1
    dim[np.where(dim==lim)] = 0
    
    cnt = 0
    for a,b in zip(y,x):
        if arr[a][b] == 0:
            one_move(out, a, b, n)
            cnt += 1
            #print_cuc(out)
    #print_cuc(arr)
    #print_cuc(out)
    return out, cnt

def step(arr):
    out, cnt = move(arr, 1)
    total = cnt
    out, cnt = move(out, 2)
    total += cnt
    return out, total != 0
            
for i in range(1,1000):
    arr, is_move = step(arr)
    if not is_move:
        print(f'Done after {i} steps')
        break

Done after 400 steps
 > > > ... > > >
 > > > ... > > >
 > > > ... > > >
 ...
 > > v ... > > >
 > > > ... > > >
 > > > ... > > >

